# 閒聊準備工作

In [2]:
# # -*- coding: utf-8 -*-
# import sys
# import os
# reload(sys)
# sys.setdefaultencoding('utf-8')

# 斷詞

In [107]:
import jieba
import jieba.posseg as pseg
import pandas as pd

class JiebaSegmentor:

    def __init__(self, dict_path, userdict = []):
        self.__dict_path = dict_path
        self.__userdict = userdict
        self.dictionary_init()
    
    def dictionary_init(self):
        jieba.set_dictionary(self.__dict_path)
        for path in self.__userdict:
            jieba.load_userdict(path)
    
    def get_names(self, input_text):
        names=[]
        words = pseg.cut(input_text)
        print words
        for w,f in words:
            if f.lower() == 'nr':
                names.append(w)
        for name in names:
            print name.encode('utf-8')
        return names
            
    def lcut(self, input_text):
        cut_raw = jieba.lcut(input_text)
        return cut_raw
    
    def pseg_lcut(self, input_text):
        cut_raw = pseg.lcut(input_text)
        key = []
        value = []
        
        for k,v in cut_raw:
            key.append(k)
            value.append(v)
        df = pd.DataFrame({"word":key, "tag":value})
        return df

In [108]:
jieba_dict_path = "/home/charles/dataset/userdict.txt"
segmentor = JiebaSegmentor(jieba_dict_path)
df = segmentor.pseg_lcut('請問今天下雨嗎請問今天下雨嗎')
df

Building prefix dict from /home/charles/dataset/userdict.txt ...
DEBUG:jieba:Building prefix dict from /home/charles/dataset/userdict.txt ...
Loading model from cache /tmp/jieba.ubc05a47a38c222127669bf922eb0875d.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.ubc05a47a38c222127669bf922eb0875d.cache
Loading model cost 0.239 seconds.
DEBUG:jieba:Loading model cost 0.239 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


,tag,word
0,x,請問
1,t,今天
2,v,下雨
3,x,嗎
4,x,請問
5,t,今天
6,v,下雨
7,x,嗎


# TFIDF

In [124]:
from gensim.corpora import *
from gensim.models import TfidfModel
from datetime import datetime
from gensim import models

class TfidfGensim:

    def __init__(self, model_path, dictionary_path, stopword_path,
                 jieba_dict_path, jieba_user_dict_path_list=[]):
        self.__model_path = model_path
        self.model = None
        self.model_init()
        self.__stopword_path = stopword_path
        self.stopword_set = set()
        self.stopword_init()
        self.__dictionary_path = dictionary_path
        self.dictionary_init()
        self.jieba_dict_path = jieba_dict_path
        self.jieba_user_dict_path_list = jieba_user_dict_path_list
        self.segmentor = None
        self.jieba_init()

    def stopword_init(self):
        self.stopword_set = set()
        with open(self.__stopword_path,'r') as stopwords:
            for sw in stopwords:
                self.stopword_set.add(sw.strip('\n').decode('utf-8'))
    
    def model_init(self):
        self.model = TfidfModel.load(self.__model_path)

    def dictionary_init(self):
        self.dictionary = Dictionary.load(self.__dictionary_path)
    
    def jieba_init(self):
        self.segmentor = JiebaSegmentor(self.jieba_dict_path, self.jieba_user_dict_path_list)

    def predict(self, input_text):
        words = segmentor.pseg_lcut(input_text)
        subject_dict={}
        for index, row in words.iterrows():
#             print row['word'] + ' (' + row['tag'] + ')'
            subject_dict.update({row['word']:row['tag']})
            
        corpus = []
        for index, row in words.iterrows():
            if row['word'] not in self.stopword_set:
                corpus.append(row['word'])
            
        corpus_key = self.dictionary.doc2bow(corpus)
        corpus_tfidf = self.model[corpus_key]
        corpus_tfidf = sorted(corpus_tfidf, key=lambda item: item[1], reverse=True)
        id2token = dict(zip(self.dictionary.token2id.values(), self.dictionary.token2id.keys()))
#         result = []
        key = []
        value = []
        for i in range(len(corpus_tfidf)):
#             result.update({id2token[corpus_tfidf[i][0]]: corpus_tfidf[i][1]})
#             print "--------------------------------------"
            word = id2token[corpus_tfidf[i][0]]
            tfidf = corpus_tfidf[i][1]
#             print 'keyword: ' + word + '(' + subject_dict[word] + ') , tfidf: ' + str(tfidf)
            key.append(word)
            value.append(tfidf)
#             result.append((word, tfidf))
        
        df = pd.DataFrame({"word":key, "tfidf":value})
        return df

#         return result
#             try:
#                 w2v_predict(w2v_model, word)
#             except KeyError as er:
#                 print 'no synonyms'

In [41]:
tfidf_model_path = "/home/charles/dataset/tfidf_model/tfidf.model"
tfidf_dictionary_path = "/home/charles/dataset/model/tfidf_corpus_dict"
stopwords_path = "/home/charles/dataset/stopwords.txt"
jieba_dict_path = "/home/charles/dataset/userdict.txt"

tg = TfidfGensim(tfidf_model_path, tfidf_dictionary_path, 
                   stopwords_path, 
                   jieba_dict_path)
tg.predict('請問今天下雨嗎')

Building prefix dict from /home/charles/dataset/userdict.txt ...
DEBUG:jieba:Building prefix dict from /home/charles/dataset/userdict.txt ...
Loading model from cache /tmp/jieba.ubc05a47a38c222127669bf922eb0875d.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.ubc05a47a38c222127669bf922eb0875d.cache
Loading model cost 0.246 seconds.
DEBUG:jieba:Loading model cost 0.246 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


請問 (x)
今天 (t)
下雨 (v)
嗎 (x)
--------------------------------------
keyword: 請問(x) , tfidf: 0.8771051416986487
--------------------------------------
keyword: 今天(t) , tfidf: 0.3464878958700677
--------------------------------------
keyword: 下雨(v) , tfidf: 0.33261495519793777


# Word to vector

In [125]:
from gensim import models

class WordToVecGensim:

    def __init__(self, model_path):
        self.__model_path = model_path
        self.model_init()

    def model_init(self):
        self.model = models.Word2Vec.load(self.__model_path)

    def predict(self, input_text):

        if type(input_text) is str:
            input_text = input_text.decode('utf-8')

        print 'synonyms : '
        key = []
        value = []
        try:
            res = self.model.wv.most_similar(input_text)
            for item in res:
                key.append(item[0])
                value.append(item[1])
#             result.append((word, tfidf))

        except KeyError as er:
            print 'no synonyms'

        df = pd.DataFrame({"word":key, "cof":value})
        return df

In [42]:
w2v_model_path = "/home/charles/dataset/word2vec_taiwan.model"
del w2v
w2v = WordToVecGensim(w2v_model_path)

In [45]:
input_text = '關雲長'
print 'input_text: ' + input_text
res = w2v.predict(input_text)
res = list(res)
for index,item in enumerate(res):
    if index == 5:
        break

    if item[1] < 0.15:
        break
    print item[0] + ': ' + str(item[1])

input_text: 關雲長
synonyms : 
張豐毅: 0.746361374855
湯鎮業: 0.737364172935
銅雀臺: 0.735068023205
薛剛: 0.731519162655
劉家輝: 0.728982448578


# 問句分類

# 閒聊

In [109]:
jieba_dict_path = "/home/charles/dataset/userdict.txt"
del segmentor
segmentor = JiebaSegmentor(jieba_dict_path)

In [126]:
tfidf_model_path = "/home/charles/dataset/tfidf_model/tfidf.model"
tfidf_dictionary_path = "/home/charles/dataset/model/tfidf_corpus_dict"
stopwords_path = "/home/charles/dataset/stopwords.txt"
jieba_dict_path = "/home/charles/dataset/userdict.txt"

del tg
tg = TfidfGensim(tfidf_model_path, tfidf_dictionary_path, 
                   stopwords_path, 
                   jieba_dict_path)

In [127]:
w2v_model_path = "/home/charles/dataset/word2vec_taiwan.model"
del w2v
w2v = WordToVecGensim(w2v_model_path)

In [113]:
ww = '今天天氣好晴朗,路邊野花香'
df = segmentor.pseg_lcut(ww)
df
# for index, row in df.iterrows():
#     print row['word']+ '(' + row['tag']+ ')'

,tag,word
0,t,今天
1,x,天氣
2,a,好
3,a,晴朗
4,x,","
5,s,路邊
6,n,野花
7,ns,香


In [128]:
ww = '今天天氣好晴朗,今天天氣好晴朗,路邊野花香,晴朗,晴朗,晴朗'
df = tg.predict(ww)
df
# for item in res:
#     print item[0] + '(' + str(item[1]) + ')'

Building prefix dict from /home/charles/dataset/userdict.txt ...
DEBUG:jieba:Building prefix dict from /home/charles/dataset/userdict.txt ...
Loading model from cache /tmp/jieba.ubc05a47a38c222127669bf922eb0875d.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.ubc05a47a38c222127669bf922eb0875d.cache
Loading model cost 0.255 seconds.
DEBUG:jieba:Loading model cost 0.255 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


,tfidf,word
0,0.915588,晴朗
1,0.303557,天氣
2,0.152305,野花
3,0.143543,今天
4,0.121165,路邊
5,0.076593,香
6,0.072126,好


In [129]:
df = w2v.predict('今天')
df
# for item in res:
#     print item[0] + '(' + str(item[1]) + ')'

synonyms : 


,cof,word
0,0.701232,今日
1,0.652387,如今
2,0.648781,現在
3,0.531828,那時
4,0.514310,現今
5,0.512732,昨天
6,0.485135,當今
7,0.480213,原來
8,0.469453,這裡
9,0.460246,這兒


In [ ]:
def chat(input_text):
    # 斷詞
    cut_text = segmentor.pseg_lcut(input_text)
    # 問句分類
    # 找關鍵字
    keyword = tg.predict(input_text)
    